In [2]:
# !pip install vncorenlp
# !git clone https://github.com/vncorenlp/VnCoreNLP.git

In [3]:
# !wget https://github.com/vncorenlp/VnCoreNLP/archive/refs/tags/v1.2.zip
# !unzip v1.2.zip


In [4]:
# from vncorenlp import VnCoreNLP
# import os

# # Khởi động VnCoreNLP
# rdrsegmenter = VnCoreNLP(os.path.join("/kaggle/working/VnCoreNLP-1.2/VnCoreNLP-1.2.jar"), annotators="wseg,pos,ner,parse", max_heap_size='-Xmx2g')

# # Hàm tách từ sử dụng VnCoreNLP
# def word_segment(text):
#     return ' '.join(rdrsegmenter.tokenize(text)[0])


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pandas as pd
import os
from tqdm.auto import tqdm
tqdm.pandas()

In [5]:
data = pd.read_csv('/kaggle/input/data-dl-tk2/df_final_a.csv')
data = data.dropna().reset_index()
# data['cleaned_segmented'] = data['cleaned_old_data'].progress_apply(word_segment)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
# X_train, X_test, y_train, y_test = train_test_split(data['cleaned_segmented'], data['label'], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_old_data'], data['label'], test_size=0.2, random_state=42)

# Chuyển đổi văn bản thành vector TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [6]:
# Huấn luyện mô hình Naive Bayes
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test_tfidf)

# Đánh giá mô hình
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.62      0.69      0.65      2380
           1       0.55      0.47      0.51      2315
           2       0.54      0.53      0.54      2415
           3       0.66      0.55      0.60      2416
           4       0.60      0.64      0.62      2257
           5       0.65      0.75      0.69      2442

    accuracy                           0.60     14225
   macro avg       0.60      0.60      0.60     14225
weighted avg       0.60      0.60      0.60     14225



In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

clf = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
clf.fit(X_train_tfidf, y_train)

# Dự đoán và đánh giá mô hình
y_pred = clf.predict(X_test_tfidf)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.7476    0.7378    0.7427      2380
           1     0.6924    0.6156    0.6517      2315
           2     0.6584    0.6513    0.6549      2415
           3     0.7890    0.8108    0.7998      2416
           4     0.7125    0.7523    0.7319      2257
           5     0.8736    0.9169    0.8947      2442

    accuracy                         0.7487     14225
   macro avg     0.7456    0.7475    0.7459     14225
weighted avg     0.7466    0.7487    0.7470     14225



In [8]:
# pip install xgboost

import xgboost as xgb

# Chuyển đổi nhãn thành mảng numpy
y_train_np = y_train.values
y_test_np = y_test.values

# Tạo DMatrix cho XGBoost
dtrain = xgb.DMatrix(X_train_tfidf, label=y_train_np)
dtest = xgb.DMatrix(X_test_tfidf, label=y_test_np)

# Thiết lập tham số cho XGBoost
params = {
    'objective': 'multi:softmax',
    'num_class': 6,
    'eval_metric': 'merror',
    'max_depth': 6,
    'eta': 0.1,
    'seed': 42
}

# Huấn luyện mô hình
bst = xgb.train(params, dtrain, num_boost_round=100)

# Dự đoán
y_pred = bst.predict(dtest)

# Đánh giá mô hình
print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

           0     0.8052    0.7971    0.8011      2380
           1     0.7117    0.6281    0.6673      2315
           2     0.7373    0.7251    0.7311      2415
           3     0.8899    0.8998    0.8948      2416
           4     0.7690    0.8631    0.8134      2257
           5     0.9628    0.9758    0.9693      2442

    accuracy                         0.8160     14225
   macro avg     0.8127    0.8148    0.8128     14225
weighted avg     0.8142    0.8160    0.8142     14225



In [9]:
# Cài đặt các thư viện cần thiết
# pip install transformers torch scikit-learn pandas

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torch.nn.utils import clip_grad_norm_

# Thiết lập thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

# Tải tokenizer và model PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=6)
model.to(device)

# Tiền xử lý dữ liệu cho PhoBERT
def encode_texts(texts, tokenizer, max_length=256):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

print("Bắt đầu mã hóa dữ liệu...")
X_train_enc = encode_texts(X_train, tokenizer)
X_test_enc = encode_texts(X_test, tokenizer)
print("Hoàn tất mã hóa dữ liệu.")

# Chuyển đổi nhãn thành tensor
y_train_tensor = torch.tensor(y_train.values)
y_test_tensor = torch.tensor(y_test.values)

# Tạo DataLoader
batch_size = 64  # Giảm batch size để tránh vấn đề về bộ nhớ GPU

train_dataset = TensorDataset(X_train_enc['input_ids'], X_train_enc['attention_mask'], y_train_tensor)
test_dataset = TensorDataset(X_test_enc['input_ids'], X_test_enc['attention_mask'], y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Huấn luyện mô hình
optimizer = AdamW(model.parameters(), lr=1e-5)  # Giảm learning rate từ 2e-5 xuống 1e-5

num_epochs = 10  # Tăng số epoch để mô hình có thời gian học nhiều hơn
total_steps = len(train_loader) * num_epochs

# Thiết lập scheduler để điều chỉnh learning rate
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Hàm tính độ chính xác
def compute_accuracy(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(preds_flat == labels_flat) / len(labels_flat)

# Early Stopping parameters
patience = 2
best_accuracy = 0
epochs_no_improve = 0

print("Bắt đầu huấn luyện mô hình...")
for epoch in range(num_epochs):
    print(f"\n===== Epoch {epoch + 1} / {num_epochs} =====")
    model.train()
    total_loss = 0
    total_accuracy = 0

    for step, batch in enumerate(train_loader):
        model.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        # Tính độ chính xác trên batch hiện tại
        logits = logits.detach().cpu().numpy()
        label_ids = labels.cpu().numpy()
        batch_accuracy = compute_accuracy(logits, label_ids)
        total_accuracy += batch_accuracy

        loss.backward()
        # Clip gradients để tránh gradient explosion
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        if step % 10 == 0 and step != 0:
            avg_loss = total_loss / (step + 1)
            avg_accuracy = total_accuracy / (step + 1)
            print(f"  Batch {step}/{len(train_loader)} - Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}")

    avg_epoch_loss = total_loss / len(train_loader)
    avg_epoch_accuracy = total_accuracy / len(train_loader)
    print(f"\nKết quả Epoch {epoch + 1}:")
    print(f"  Loss trung bình: {avg_epoch_loss:.4f}")
    print(f"  Độ chính xác trung bình: {avg_epoch_accuracy:.4f}")

    # Đánh giá và kiểm tra Early Stopping
    if avg_epoch_accuracy > best_accuracy:
        best_accuracy = avg_epoch_accuracy
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pt')
        print("  -> Cập nhật mô hình tốt nhất.")
    else:
        epochs_no_improve += 1
        print(f"  -> Không cải thiện được trong {epochs_no_improve} epoch.")
        if epochs_no_improve >= patience:
            print("Early stopping được kích hoạt.")
            break

print("Hoàn tất huấn luyện.")

Sử dụng thiết bị: cuda


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Bắt đầu mã hóa dữ liệu...
Hoàn tất mã hóa dữ liệu.


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Bắt đầu huấn luyện mô hình...

===== Epoch 1 / 10 =====
  Batch 10/889 - Loss: 1.7887, Accuracy: 0.1832
  Batch 20/889 - Loss: 1.7748, Accuracy: 0.2106
  Batch 30/889 - Loss: 1.7543, Accuracy: 0.2450
  Batch 40/889 - Loss: 1.7401, Accuracy: 0.2645
  Batch 50/889 - Loss: 1.7172, Accuracy: 0.2953
  Batch 60/889 - Loss: 1.6943, Accuracy: 0.3145
  Batch 70/889 - Loss: 1.6716, Accuracy: 0.3334
  Batch 80/889 - Loss: 1.6501, Accuracy: 0.3492
  Batch 90/889 - Loss: 1.6249, Accuracy: 0.3656
  Batch 100/889 - Loss: 1.6007, Accuracy: 0.3793
  Batch 110/889 - Loss: 1.5796, Accuracy: 0.3877
  Batch 120/889 - Loss: 1.5590, Accuracy: 0.3972
  Batch 130/889 - Loss: 1.5396, Accuracy: 0.4061
  Batch 140/889 - Loss: 1.5215, Accuracy: 0.4145
  Batch 150/889 - Loss: 1.5020, Accuracy: 0.4240
  Batch 160/889 - Loss: 1.4868, Accuracy: 0.4299
  Batch 170/889 - Loss: 1.4706, Accuracy: 0.4381
  Batch 180/889 - Loss: 1.4538, Accuracy: 0.4455
  Batch 190/889 - Loss: 1.4401, Accuracy: 0.4515
  Batch 200/889 - Loss

NameError: name 'os' is not defined

In [11]:
# Lưu mô hình và tokenizer sử dụng save_pretrained
save_directory = 'saved_phobert_model'

# Tạo thư mục lưu nếu chưa tồn tại
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Lưu mô hình
model.save_pretrained(save_directory)

# Lưu tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Mô hình và tokenizer đã được lưu tại {save_directory}")

# Đánh giá mô hình trên tập kiểm tra
print("\nBắt đầu đánh giá mô hình trên tập kiểm tra...")
model.load_state_dict(torch.load('/kaggle/working/best_model.pt'))  # Tải mô hình tốt nhất
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        y_pred.extend(predictions.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

print("Hoàn tất đánh giá.")

print("\nBáo cáo đánh giá:")
print(classification_report(y_true, y_pred, digits=5))


Mô hình và tokenizer đã được lưu tại saved_phobert_model

Bắt đầu đánh giá mô hình trên tập kiểm tra...


/tmp/ipykernel_30/2468469313.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/best_model.pt'))  # Tải mô hình tốt nhất


Hoàn tất đánh giá.

Báo cáo đánh giá:
              precision    recall  f1-score   support

           0    0.85888   0.83109   0.84476      2380
           1    0.79031   0.83844   0.81367      2315
           2    0.79199   0.76149   0.77644      2415
           3    0.93977   0.96233   0.95092      2416
           4    0.90081   0.88923   0.89498      2257
           5    0.98444   0.98444   0.98444      2442

    accuracy                        0.87831     14225
   macro avg    0.87770   0.87784   0.87753     14225
weighted avg    0.87831   0.87831   0.87808     14225



In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Đường dẫn tới thư mục đã lưu mô hình
save_directory = '/kaggle/working/saved_phobert_model'

# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_directory)

# Tải mô hình
model = AutoModelForSequenceClassification.from_pretrained(save_directory, num_labels=6)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("Mô hình và tokenizer đã được tải lại thành công.")

# Hàm dự đoán
def predict(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()
    return predicted_class

# Ví dụ dự đoán
sample_comment = "hàng không sử dụng được, tốn tiền mua nhưng về vứt xó không sử dụng"
predicted_label = predict(sample_comment)
print(f"Bình luận: '{sample_comment}' được phân loại vào lớp: {predicted_label}")


Mô hình và tokenizer đã được tải lại thành công.
Bình luận: 'hàng không sử dụng được, tốn tiền mua nhưng về vứt xó không sử dụng' được phân loại vào lớp: 0
